# Упражнение 2. Качество данных

## Utils

In [6]:
import time
from datetime import date, datetime, timedelta
import pandas as pd
from typing import Optional, List
import warnings

warnings.filterwarnings('ignore', message="registration of accessor <class '__main__.CustomAccessor'>")
warnings.filterwarnings('ignore', message="registration of accessor <class '__main__.SCD'>")
warnings.simplefilter(action='ignore', category=FutureWarning)

## Задание 2.0

В упражнении 1 уже скачивали данные об учебных планах. Убедитесь в том, что эти данные у вас сохранились.

In [1]:
%ls data

academic_plan_id.csv
academic_plan_id.csv.dvc
academic_plans.csv
academic_plans.csv.dvc
academic_plans_in_field_of_study.csv
academic_plans_in_field_of_study.csv.dvc
all_details.json
all_details.json.dvc
editors.csv
editors.csv.dvc
fields_of_study.csv
fields_of_study.csv.dvc
structural_units.csv
structural_units.csv.dvc


## Задание 2.1 Получение данных

In [4]:
from utils.data_collector import OPITMODataCollector

data_collector = OPITMODataCollector()

Идентификаторы всех учебных планов, имеющихся в op.itmo.ru

In [3]:
# Get new ids from API:
# ids = data_collector.get_academic_plans_ids()
# ids = list(ids)

# User stored ids in DVC:
ids = data_collector.get_data('data/academic_plan_id')['academic_plan_id'].tolist()

Found 60 pages

In [7]:
df_aps = pd.DataFrame(ids, columns=['academic_plan_id'])
df_aps.custom.to_csv_with_time('academic_plan_id')
df_aps

,academic_plan_id
0,6796
1,6797
2,6798
3,6799
4,6800
...,...
592,7501
593,7502
594,7503
595,7504


In [8]:
# Save collected data in DVC
# data_collector.save_data(df_aps, 'academic_plan_id')

# Or Local
# df_aps.to_csv(f'data/ids_{time.time()}.csv')

По этим планам получить детальное описание

In [10]:
# Update data via API:
all_details = data_collector.get_academic_plan_details(list(ids))

# User stored data in DVC:
# all_details = data_collector.get_data('data/all_details', file_type='json')

                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                |||||||||||||||||||||||||||||||||||||||||||||||

In [11]:
from utils.json_dict_converter import JsonDictConverter

# Save collected data in DVC
# data_collector.save_data(all_details, 'all_details')

# Or Local
JsonDictConverter().to_json(all_details, json_name='data/all_details', timed=True)

Медленно меняющиеся измерения (от англ. Slowly Changing Dimensions, SCD) — механизм отслеживания изменений в данных измерения. [Выделяют несколько типов SCD2. ](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%BB%D0%B5%D0%BD%D0%BD%D0%BE_%D0%BC%D0%B5%D0%BD%D1%8F%D1%8E%D1%89%D0%B5%D0%B5%D1%81%D1%8F_%D0%B8%D0%B7%D0%BC%D0%B5%D1%80%D0%B5%D0%BD%D0%B8%D0%B5).

Реализуйте механизм SCD2. Добавилась строка и столбцы с датой вступления изменения в силу (столбце с TRUE/FALSE необязательно)



Мы решили добавлять колонку со статусом, чтобы отслеживать **полностью удаленные** значения.

In [ ]:
@pd.api.extensions.register_dataframe_accessor('scd')
class SCD:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj


    @staticmethod
    def get_date_str(day: datetime.date) -> str:
        day = date.today() if day is None else day
        return day.strftime('%d_%m_%Y')


    def add_effective_date(self, date: datetime.date, time_col: str = 'effective_date', inplace: bool = False) -> Optional[pd.DataFrame]:
        date = self.get_date_str(date)
        if inplace == False:
            df_copy = self._obj.copy()
            df_copy[time_col] = date
            return df_copy
        self._obj[time_col] = date


    @classmethod
    def update_effective_date(cls, df: pd.DataFrame, date: datetime.date = None, time_col: str = 'effective_date') -> pd.DataFrame:
        date = cls.get_date_str(date)
        df[time_col] = df[time_col].fillna(date)

        return df


    def add_status(self, status_col: str = 'status', inplace: bool = False) -> Optional[pd.DataFrame]:
        if inplace == False:
            df_copy = self._obj.copy()
            df_copy[status_col] = True
            return df_copy
        self._obj[status_col] = True


    @staticmethod
    def update_status(df: pd.DataFrame, deleted_ids: List[int] = [], id_col: str = 'id', status_col: str = 'status') -> pd.DataFrame:
        df[status_col] = False
        df[status_col] = df.groupby(id_col).cumcount(ascending=False) == 0
        df.loc[df[id_col].isin(deleted_ids), status_col] = False

        return df


    def update(self, df: pd.DataFrame, id_col: str = 'id', time_col: str = 'effective_date', status_col: str = 'status', inplace: bool = False) -> Optional[pd.DataFrame]:
        deleted_ids = (set(self._obj[id_col].tolist()) - set(df[id_col].tolist()))

        check_columns = list(set(self._obj.columns) - set([time_col, status_col]))
        df = pd.concat([self._obj, df]).drop_duplicates(subset=check_columns).reset_index(drop=True)

        df = self.update_effective_date(df)
        df = df.sort_values(by=[id_col, time_col])
        df = self.update_status(df, deleted_ids=deleted_ids)

        if inplace == False:
            return df
        self._obj = df

In [ ]:
def get_data_from_google_sheets(key: str, sheet: str) -> pd.DataFrame:
    url = f'https://docs.google.com/spreadsheet/ccc?key={key}&output=xlsx'
    df = pd.read_excel(url, sheet_name=sheet)
    return df

In [ ]:
key = '1Ad9xWwEv25ESNQEly8p52nErAnHzZeA0wwvmfUHp2lg'
df_v1 = get_data_from_google_sheets(key, 'data_v1')
df_v2 = get_data_from_google_sheets(key, 'data_v2')

In [ ]:
df_v1

,id,field_1,field_2
0,1,a,3
1,2,b,4
2,3,c,5


In [ ]:
df_v2

,id,field_1,field_2
0,1,a,2
1,2,b,4
2,4,d,6


In [ ]:
df_v1.scd.add_effective_date(date=datetime.now()-timedelta(1), inplace=True)
df_v1.scd.add_status(inplace=True)
df = df_v1.scd.update(df_v2)

df

,id,field_1,field_2,effective_date,status
0,1,a,3,21_10_2023,False
3,1,a,2,22_10_2023,True
1,2,b,4,21_10_2023,True
2,3,c,5,21_10_2023,False
4,4,d,6,22_10_2023,True


## Задание 2.2 Качество данных

Необходимо выбрать не менее трех характеристик качества данных и сделать профилирование данных, которое позволит измерить эти характеристики. Например, полнота и визуализация, на которой видны пропущенные значения.

В 1996 году Ричард Уанг и Дайана Стронг предложили модель оценки качества данных по следующим 15 параметрам (измерениям) их восприятия потребителями
* Качество данных как таковых:
1. точность;
2. объективность;
3. убедительность;
4. репутация источника.
* Контекстуальное качество данных:
1. полезность;
2. релевантность;
3. актуальность;
4. полнота;
5. достаточность объема данных.
* Репрезентативность:
1. интерпретируемость;
2. понятность;
3. логичность представления;
4. лаконичность представления.
* Наличие подтверждения качества данных:
1. доступность;
2. безопасность доступа и защита от несанкционированного доступа.

В этой работе рассмотрим следующие:
1. точность (accuracy),
2. завершенность (completeness),
3.

## Задание 2.3 Метаданные

Подготовьте наиболее полные метаданные по данным, полученным в задании 2.1

# Критерии и баллы

Распределение исходя из 30 баллов.



*   2.0 - 1 балл
*   2.1 - 8 баллов
*   2.2 - 12 баллов
*   2.3 - 9 баллов

